In [79]:
## EDDI processing code - monthly_eddi_pctl.py  
## Used by monthly_eddi_pctl.py and offmonthly_eddi_pctl.py
## Last updated: 2024-08-12
## Keith Eggleston and Kent Schneider

import sys, requests, calendar, json
from datetime import datetime, timedelta
import numpy as np
from scipy import stats
#from local_defs import directories

# Directory for input files
indir = "indir/"
#indir = directories["indir"]

area_names = {
	'Gridpoints': {'code': 1}
}


def process_eddi3 (ryear, rmonth, rday):
	# check if a file exists for requested day (last day of month), otherwise last day available (within 10 days)
	check_date = datetime(ryear, rmonth, rday)
	last_to_check = check_date - timedelta(days=10)
	use_date = None
	while check_date >= last_to_check:
		yyyymmdd = check_date.strftime("%Y%m%d")
		filename = "EDDI_ETrs_02mn_{}.asc".format(yyyymmdd)
		print ('Checking for',filename)
		url = "https://downloads.psl.noaa.gov/Projects/EDDI/CONUS_archive/data/{0}/{1}".format(check_date.year, filename)
		response = requests.get(url)
		if response.ok:
			use_date = check_date
			break
		check_date = check_date - timedelta(days=1)

	if not use_date:
		print ('No data within 10 days of requested date; exiting...')
		sys.exit()
	else:
		print ("Current year data file is ", use_date.strftime("%Y-%m-%d"))
		ryear = use_date.year
		rmonth = use_date.month
		rday = use_date.day

	#####################################
	### load EDDI data from files
	######################################

	today = datetime.today()
	gridpt_eddi = {
		"data": {},
		"date": str(today),
		"data_thru": "{0}".format(use_date.strftime("%Y-%m-%d"))
	}
	for area in area_names.keys():
		gridpt_eddi["data"][area] = {}

	#basin_file = open('{}ma_basin_grids.json'.format(indir), 'r')
	#ma_basin_grids = json.load(basin_file)
	#region_file = open('{}ma_region_grids.json'.format(indir), 'r')
	#ma_region_grids = json.load(region_file)
	ny_file = open('{}ny_grids.json'.format(indir), 'r')
	ny_grids = json.load(ny_file)


	#for timescale in ['02mn', '01mn']:	#order is important for populating dictionaries below
	for timescale in ['01mn']:
		print (timescale,'- Loading data from files')
		area_vals = {}
		for area in area_names.keys():
			area_vals[area] = {}

		for y in range(1980, ryear+1):
			# period with bad forcing re: Mike Hobbins 
			if (y == 2022 and rmonth >= 8) or (y == 2023 and rmonth <= 7):
				print ('Suspect data; skipping {0}-{1}'.format(y,rmonth))
				continue
			if rmonth == 2 and rday == 29 and not calendar.isleap(y):
				tday = 28
			else:
				tday = rday
			yyyymmdd = "%s%02d%02d" % (y,rmonth,tday)

			#print (yyyymmdd)
			filename = "EDDI_ETrs_{0}_{1}.asc".format(timescale,yyyymmdd)
			url = "https://downloads.psl.noaa.gov/Projects/EDDI/CONUS_archive/data/{0}/{1}".format(y,filename)
			response = requests.get(url)
			if response.ok:
				ifile = response.text
			else:
				print (filename,"; bad status code:",response.status_code,'; skipping ...')
				continue

			linesOfFile = ifile.split('\n')

			#information lines
			nlon = int(linesOfFile[0].split()[-1])
			nlat = int(linesOfFile[1].split()[-1])
			ll_lon = float(linesOfFile[2].split()[-1])
			ll_lat = float(linesOfFile[3].split()[-1])
			res = float(linesOfFile[4].split()[-1])
			miss = float(linesOfFile[5].split()[-1])

			#check for errors or changes to file header info
			if nlon != 464:
				print ('Wrong nlon', nlon)
			if nlat != 224:
				print ('Wrong nlat', nlat)
			if ll_lon != -125.0:
				print ('Wrong ll_lon', ll_lon)
			if ll_lat != 25.0:
				print ('Wrong ll_lat', ll_lat)
			if res != 0.125:
				print ('Wrong res', res)

			#lines of data
			data_list = []
			for line in linesOfFile[6:]:
				if len(line) > 0:
					data_list.append(line.split())

			#check for correct shape
			if len(data_list) != 224:
				print ('Wrong number of lines od data', len(data_list))
			if len(data_list[0]) != 464:
				print ('Wrong number of columns of data', len(data_list[0]))

			#extract data for each grid points in specified REGION
			for region in ny_grids.keys():
				pt = 0
				for coor in ny_grids[region]:
					if not pt in area_vals[region].keys():
						area_vals[region][pt] = []
					val = float(data_list[coor[3]][coor[2]])	#x,y coordinates in grid
					area_vals[region][pt].append([yyyymmdd,val * -1])	#reverse sign so negative values are dry, instead of wet
					pt += 1

			#extract data for each grid points in specified BASIN

		# perform calculations using retrieved data
		for region in ny_grids.keys():
			pt = 0
			dct = {}
			for coor in range(len(area_vals[region])):
				hist = area_vals[region][coor][:-1]
				for dataset in hist:
					#print(dataset)
					if f'{dataset[0][:4]}-{dataset[0][4:6]}-{dataset[0][6:]}' in dct.keys():
						dct[f'{dataset[0][:4]}-{dataset[0][4:6]}-{dataset[0][6:]}'] += dataset[1]
					else:
						#print(f'{dataset[0][:4]}-{dataset[0][4:6]}-{dataset[0][6:]} not in dct')
						dct[f'{dataset[0][:4]}-{dataset[0][4:6]}-{dataset[0][6:]}'] = dataset[1]
					#print(dct.keys())
					#print(dct)
			for item in dct:
				#print(item)
				#print(len(area_vals[region]))
				dct[item] = dct[item] / len((area_vals[region]))
				#print(dct)
			return(dct)


In [82]:
# get_all_data(end_date)

import datetime as dt
def get_all_data(end_date):
    data = {}
    year = 1980
    start_date = dt.datetime(year, 1, 1)
    # Iterating through each day of the year
    #91 IS APRIL 1
    for i in range(91,274):
        if i == 59:
            continue
        current_date = start_date + dt.timedelta(days=i)
        data.update(process_eddi3(end_date, current_date.month, current_date.day))
        print(f"{current_date.strftime('%m/%d')} completed")
        outfilename = '%sall_data_saved.json'%(outdir)
        outfile = open(outfilename, "w")
        json.dump(data, outfile)
        outfile.close()
    ## Write regional results to JSON file - 'regional_eddi_[year]-[month].json'
    outfilename = '%sall_data_saved.json'%(outdir)
    outfile = open(outfilename, "w")
    json.dump(data, outfile)
    outfile.close()
    return data

In [83]:
# graph_comparison(graph, date, start_date, end_date, window, plotting, percentile, scope)
import matplotlib as plt
import datetime as dt
import numpy as np

def sortkey2(e):
    return e[1]

def daily_no_avg(graph, start, date, end_date, start2, scope):
    # returns list of that week average over all years
    date_data = []
    for date in list(graph.keys()):
        date_data.append([dt.datetime(int(date[0:4]),int(date[5:7]),int(date[8:10])), graph[date]])
    total_data = []
    # just take first day and next first day
    while start.year < int(end_date):
        lst2 = []
        accum = 0
        min = 1000000
        for i in range(len(date_data)):
            if date_data[i][0] >= start and date_data[i][0] <= start + dt.timedelta(days=scope):
                lst2.append(date_data[i])
                accum += date_data[i][1]
        if len(lst2) >= 2:
            total_data.append([start2.year, accum/len(lst2)])
        start = dt.datetime(start.year + 1, start.month, start.day)
        start2 = dt.datetime(start2.year + 1, start2.month, start2.day)
    return total_data


def graph_comparison(graph, date, start_date, end_date, window, plotting, percentile, scope):
    if len(date) == 5:
        start = dt.datetime(int(start_date), int(date[0:2]), int(date[3:5]))
        start2 = start + dt.timedelta(weeks=window)
        end2 = start2 + dt.timedelta(days=scope)
    first_set = daily_no_avg(graph[0], start, date, end_date, start, scope)
    second_set = daily_no_avg(graph[1], start2, date, end_date, start, scope)
    plottable = []
    for i in first_set:
        for j in second_set:
             if j[0] == i[0]:
                  if i[1] != 0:
                    plottable.append([i[0], j[1] - i[1]])
    dct = {}
    plottable.sort(key=sortkey2)
    for i in range(len(plottable)):
        if plotting:
            dct[(str(plottable[i][0]))[2:4]] = plottable[i][1]
        else:
            dct[(str(plottable[i][0]))] = plottable[i][1]
    if plotting:
        plt.bar(dct.keys(), dct.values(), color = "green", width = 0.8)
        plt.xlabel("Year")
        plt.ylabel("Value")
        plt.title("Streamflow (%s) versus period after" % (date))
        plt.show()
    else:
        if len(dct) > 0:
            if percentile != 0:
                a = list(dct.values())
                a = np.array(a)
                p = np.percentile(a, percentile)  # return 50th percentile, i.e. median.
                return p
            else:
                # [mean, median, interquartile range,standard deviation]
                b = list(dct.values())
                b = np.array(b)

                # Don't need mean, median etc, but is possible
                return [list(dct.values()), 0, 0, 0, 0,start.strftime("%m/%d"), end2.strftime("%m/%d"), list(dct.keys())]
                #return [list(dct.values()), np.mean(b), np.median(b), np.percentile(b, 75) - np.percentile(b, 25), np.std(b),start.strftime("%m/%d"), end2.strftime("%m/%d"), list(dct.keys())]
        else:
            return False

In [87]:
# using_all_weeks(date, end_date, percent, scope, window)

def sortkey(e):
    return e[0]


def using_all_weeks(date, end_date, percent, scope, window):
    start_date = 1980


    # IF YOU ARE GATHERING RAW DATA AND NOT USING FROM A SOURCE, RUN get_all_data(end_date) AS LISTED BELOW
    #graph = get_all_data(end_date)

    # otherwise, file can be loaded here, change Long Island2.json to name of file
    graph = json.load(open('{}Long Island2.json'.format(outdir), 'r'))
    date = dt.datetime(start_date, int(date[0:2]), int(date[3:5]))
    total_data = []
    date_data = []
    for week in range(53):
        date_data.append({})
    for stat in graph:
        new_date = dt.datetime(int(stat[0:4]),int(stat[5:7]),int(stat[8:10]))
        date_data[((new_date.timetuple().tm_yday - 1) // 7)][stat] = graph[stat]

    for week in range(52):
        date2 = date + dt.timedelta(weeks=week)
        # investigate looking across year
        #Convert to string
        mo = str(date2.month)
        da = str(date2.day)
        if len(mo) == 1:
            mo = "0" + mo
        if len(da) == 1:
            da = '0' + da
        temp1 = {}
        temp2 = {}
        temp1 = {**temp1,**date_data[week]}
        if week + window < 52:
            temp2 = {**temp2,**date_data[week + window]}
        else:
            temp2 = {**temp2,**date_data[(52 - week + window)]}
        if scope // 7 > 1:
            for additional_week in range((scope // 7) + 1):
                if additional_week + week < 53:
                    temp1 = {**temp1, **date_data[week + additional_week]}
                else:
                    temp1 = {**temp1, **date_data[week + additional_week - 52]}
                if additional_week + week + (scope // 7) < 52:
                    temp2 = {**temp2, **date_data[week + additional_week + (scope // 7)]}
                else:
                    temp2 = {**temp2, **date_data[week + additional_week - 52 + (scope // 7)]}
        # COMBINE NECESSARY WEEKS FOR ANALYSIS
        info = (graph_comparison([temp1, temp2], f"{mo}-{da}", start_date, end_date, window, False, 0, scope))
        if info:
            for i in range(len(info[0])):
                total_data.append([info[0][i],f"{info[7][i]}/{(info[5])} - {info[7][i]}/{info[6]}"])
    total_data.sort(key=sortkey)
    '''
    print(str(percent) + "%")
    for i in range(int(len(total_data) / (100 / percent))):
        print(f'{total_data[i][0][0]:.2f}: ({total_data[i][0][1]:.2f}), {total_data[i][1]}')
    '''
    templst = []
    for i in (total_data[0:10]):
        templst.append(f'{i[0]:.2f}, {i[1]}')
    print(templst)
    return total_data[0:(int(int(len(total_data) / (100 / percent))))]

In [85]:
# graphing(data, end_date, percentile)

MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', "Dec"]
import matplotlib.pyplot as plt

def graphing(data, end_date, percentile):
    year_accum = []
    year_accum2 = []
    for i in range(int(end_date)-1980):
        year_accum.append(0)
        year_accum2.append(0)
    month_accum = [0,0,0,0,0,0,0,0,0,0,0,0]
    for lst in data:
            month_accum[int(lst[1][5:7]) - 1] += 1

    plt.bar(MONTHS, month_accum, color ="blue")
    plt.title(f"Evaporation: {percentile} percent between {1980}-{end_date}")
    plt.xlabel("Month")
    plt.ylabel("Number of Occurances")
    plt.show()
    if percentile == 10:
        print(f"1%: {data[int(len(data[0]) / 100)][0]:.2f}")
        print(f"2%: {data[int(len(data[0]) / 50)][0]:.2f}")
        print(f"5%: {data[int(len(data) / 20)][0]:.2f}")
        print(f"10%: {data[int(len(data) / 10)][0]:.2f}")

        for lst in data[0:int(len(data) / (10))]:
            year_accum[int(lst[1][0:4]) - int(1980)] += 1

        plt.bar((range(int(1980), int(end_date))), year_accum, color ="green")
        plt.title(f"Occurances across years: 1 percent between {1980}-{end_date}")
        plt.xlabel("Year")
        plt.ylabel("Number of Occurances")
        plt.show()

In [88]:
## Driver

# Directory for output files
outdir = "outdir/"

'''

Buffalo
-78.75 to -79.0
42.75 to 42.875

Adirondacks
-74.0 to -74.375
43.625 to 43.875

Catskills
-74.5 to -74.5
42.25 to 42.375

Long Island
-73.5 to -73.625
40.625 to 40.625

Buffalo is
[-78.75, 42.75, 370, 81],
[-78.75, 42.875, 370, 80],
[-78.875, 42.75, 369, 81],
[-78.875, 42.875, 369, 80],
[-79, 42.75, 368, 81],
[-79, 42.875, 368, 80]

Adirondacks is 
[-74.0, 43.625, 408, 74],
[-74.0, 43.75, 408, 73],
[-74.0, 43.875, 408, 72],
[-74.125, 43.625, 407, 74],
[-74.125, 43.75, 407, 73],
[-74.125, 43.875, 407, 72],
[-74.25, 43.625, 406, 74],
[-74.25, 43.75, 406, 73],
[-74.25, 43.875, 406, 72],
[-74.375, 43.625, 405, 74],
[-74.375, 43.75, 405, 73],
[-74.375, 43.875, 405, 72]

Catskills is 
[-74.5, 42.25, 404, 85],
[-74.5, 42.375, 404, 84]

LONG ISLAND IS
[-73.5, 40.625, 412, 98],
[-73.625, 40.625, 411, 98]

'''
ryear, rmonth, rday = 2023, 2, 2
percentile = 10
scope = 7
window = 4
print("4 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 4)
graphing(all_data, ryear, percentile)

print("3 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 3)
graphing(all_data, ryear, percentile)

print("2 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 2)
graphing(all_data, ryear, percentile)

print("1 week")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 1)
graphing(all_data, ryear, percentile)

#for i in all_data:
    #print(i)

4 weeks
['-3.52, 1988/05/20 - 1988/06/24', '-3.15, 2001/04/15 - 2001/05/20', '-2.98, 2001/04/08 - 2001/05/13', '-2.97, 1998/05/13 - 1998/06/17', '-2.96, 1988/05/27 - 1988/07/01', '-2.94, 1993/04/15 - 1993/05/20', '-2.75, 1998/05/06 - 1998/06/10', '-2.52, 1991/05/06 - 1991/06/10', '-2.48, 1995/07/22 - 1995/08/26', '-2.36, 1993/04/22 - 1993/05/27']
3 weeks
['-3.04, 2001/04/15 - 2001/05/13', '-3.01, 1998/05/13 - 1998/06/10', '-2.67, 1988/05/27 - 1988/06/24', '-2.62, 1988/05/20 - 1988/06/17', '-2.56, 2001/04/22 - 2001/05/20', '-2.44, 1991/05/06 - 1991/06/03', '-2.24, 1993/04/15 - 1993/05/13', '-2.23, 1993/04/22 - 1993/05/20', '-2.21, 2001/04/08 - 2001/05/06', '-2.14, 1998/05/20 - 1998/06/17']
2 weeks
['-2.44, 2001/04/22 - 2001/05/13', '-2.26, 2001/04/15 - 2001/05/06', '-2.17, 1998/05/20 - 1998/06/10', '-2.15, 2021/05/06 - 2021/05/27', '-1.98, 1991/05/06 - 1991/05/27', '-1.95, 1988/05/20 - 1988/06/10', '-1.77, 1988/05/27 - 1988/06/17', '-1.73, 1998/05/13 - 1998/06/03', '-1.69, 1997/09/09 - 

'\nonly april to september\nevaporation, rainfall, streamflow\naverage for box instead of just single coordinate-- DONE\n90-274 (april to end of sept) -- DONE\n\ndates of top 10 dates (drops) for all 3 and see for matches for all 4 weeks\nhighlight ones that match\nstart documentation\nsummarize functions\nadd dictionary of variables\n\n\nHOW TO ORGANIZE:\nList top 10 streamflow events (list 10 separate dates, list all anyway)\nList top 10 rainfall events\nList top 10 evaporation events\n\n'